In [1]:
from notebook_utils import load_tinymodel, load_tinydataset, load_saes, load_module_names
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/root/dictionary_learning/test_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/dictionary_learning/test_env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
llm = load_tinymodel()
dataset = load_tinydataset(batch_size=64, max_seq_length=128, num_datapoints=1000)
all_saes = load_saes(k=30)
module_names = load_module_names(llm)

100%|██████████| 1000/1000 [00:01<00:00, 526.14it/s]
/root/dictionary_learning/notebooks/notebook_utils.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = to

Failed to tokenize 0 tokens
Number of datapoints w/ 129 tokens: 952
Total Tokens: 0.122808M


In [23]:
target_sae_names = ['torso_1_mlp_out_transcoder', 'torso_1_res_final']
saes = [all_saes[name].to(device) for name in target_sae_names]
resid_mid = llm.torso[1].res_mlp
resid_final = llm.torso[1].res_final
mlp_out = llm.torso[1].mlp
for batch_ind, batch in enumerate(dataset):
    batch = batch.to(device)
    with torch.no_grad():
        with llm.trace(batch) as tracr:
            act_res_mid = resid_mid.output.save()
            act_res_final = resid_final.output.save()
            act_mlp_out = mlp_out.output.save()
        # Now we want to run through the saes
        transcoder = saes[0].to(device)
        sae_final = saes[1].to(device)
        # mlp_out_hat = transcoder(act_res_mid)

        # sae_final_features_hat = sae_final.encode(mlp_out_hat+act_res_mid)
        # maybe figure out a way to fold in the decoder bias?
        tr_dec = transcoder.decoder.weight
        #TODO: we removed the last weight to help w/ knowing .T and shape. 
        final_enc = sae_final.encoder.weight[:-1]
        virtual_weights = tr_dec.T @ final_enc.T

        act_res_mid = act_res_mid.to(device)
        input_features, input_acts, input_indices = transcoder.encode(act_res_mid, return_topk=True)
        mlp_out_hat = transcoder.decoder(input_features)

        output_features, output_acts, output_indices = sae_final.encode(mlp_out_hat + act_res_mid, return_topk=True)

        # Gradient equals the weights
        attribution = input_features @ virtual_weights
        break




In [28]:
input_features.shape

torch.Size([64, 129, 6144])

In [29]:
from einops import rearrange
attribution = rearrange(attribution, 'b s f -> (b s) f')

In [35]:
output_indices = rearrange(output_indices, 'b s f -> (b s) f')

In [41]:
output_indices[:, 0].shape

torch.Size([8256])

In [16]:
with torch.no_grad():
    attribution = input_features @ virtual_weights


In [8]:
input_features.shape, virtual_weights.shape

(torch.Size([64, 129, 6144]), torch.Size([6144, 6143]))

In [4]:
# We want to calculate attribution = act*gradient

# I believe this is equivalent to the weights of the activations (ignore biases)
# It'd be good to actually verify this is the case



with torch.no_grad():
    tr_dec = transcoder.decoder.weight
    #TODO: we removed the last weight to help w/ knowing .T and shape. 
    final_enc = sae_final.encoder.weight[:-1]
    virtual_weights = tr_dec.T @ final_enc.T

    act_res_mid = act_res_mid.to(device)
    input_features, input_acts, input_indices = transcoder.encode(act_res_mid, return_topk=True)
    mlp_out_hat = transcoder.decoder(input_features)

    output_features, output_acts, output_indices = sae_final.encode(mlp_out_hat + act_res_mid, return_topk=True)

    # For efficient gradient calculation, we can get the nonzero_indices of both input & output feature

    # W_input = transcoder.decoder.weight[input_indices]    

In [5]:
transcoder.decoder.weight.shape, input_indices.shape


(torch.Size([768, 6144]), torch.Size([64, 129, 30]))

In [ ]:
# def encode(self, x: torch.Tensor, return_topk: bool = False):
#     post_relu_feat_acts_BF = nn.functional.relu(self.encoder(x - self.b_dec))
#     post_topk = post_relu_feat_acts_BF.topk(self.k, sorted=False, dim=-1)

#     # We can't split immediately due to nnsight
#     tops_acts_BK = post_topk.values
#     top_indices_BK = post_topk.indices

#     buffer_BF = torch.zeros_like(post_relu_feat_acts_BF)
#     encoded_acts_BF = buffer_BF.scatter_(dim=-1, index=top_indices_BK, src=tops_acts_BK)

#     if return_topk:
#         return encoded_acts_BF, tops_acts_BK, top_indices_BK
#     else:
#         return encoded_acts_BF

# def decode(self, x: torch.Tensor) -> torch.Tensor:
#     return self.decoder(x) + self.b_dec